In [1]:
!pip install transformers datasets -q

In [2]:
from datasets import load_dataset
import torch
import numpy as np 
from transformers import AutoModel, AutoTokenizer

torch.set_grad_enabled(False)

PyTorch version 1.6.0+cu101 available.
TensorFlow version 2.3.0 available.


In [3]:
dataset = load_dataset('rotten_tomatoes')

Checking /root/.cache/huggingface/datasets/b3267cede0118f731406527d730b20a47827f64e15eaece772b81c2ec6c4e129.75128cbb0bbc33ddc2b7299d2a29d743223662ec632601fc16d2dea19bcb1072.py for additional imports.
Found main folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/rotten_tomatoes/rotten_tomatoes.py at /root/.cache/huggingface/modules/datasets_modules/datasets/rotten_tomatoes
Found specific version folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/rotten_tomatoes/rotten_tomatoes.py at /root/.cache/huggingface/modules/datasets_modules/datasets/rotten_tomatoes/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a
Found script file from https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/rotten_tomatoes/rotten_tomatoes.py to /root/.cache/huggingface/modules/datasets_modules/datasets/rotten_tomatoes/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a/rotten_tomatoes.py
Found d

In [4]:
!mkdir repr

models = ['bert-base-uncased', 'bert-large-uncased',
          'roberta-base', 'roberta-large']
splits = ['train', 'validation', 'test']

for model_name in models:
  for split in splits:
    model = AutoModel.from_pretrained(model_name).cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    tokens_pt = tokenizer(
        dataset[split]['text'],
        return_tensors='pt',
        padding='max_length',
        max_length=80
    )['input_ids'].cuda()

    data_loader = torch.utils.data.DataLoader(tokens_pt, batch_size=512)

    repr = []
    for review in data_loader:
      _, pooled = model(review)
      repr.append(pooled)

    representation = torch.cat(repr).cpu().numpy()

    print(model_name, split, representation.shape)
    np.save(f'repr/{model_name}_{split}', representation)

bert-base-uncased train (8530, 768)
bert-base-uncased validation (1066, 768)
bert-base-uncased test (1066, 768)
bert-large-uncased train (8530, 1024)
bert-large-uncased validation (1066, 1024)
bert-large-uncased test (1066, 1024)
roberta-base train (8530, 768)
roberta-base validation (1066, 768)
roberta-base test (1066, 768)
roberta-large train (8530, 1024)
roberta-large validation (1066, 1024)
roberta-large test (1066, 1024)


In [6]:
!zip -r repr.zip repr

  adding: repr/ (stored 0%)
  adding: repr/bert-base-uncased_test.npy (deflated 10%)
  adding: repr/bert-base-uncased_validation.npy (deflated 10%)
  adding: repr/roberta-base_test.npy (deflated 8%)
  adding: repr/roberta-base_train.npy (deflated 8%)
  adding: repr/bert-large-uncased_train.npy (deflated 10%)
  adding: repr/bert-large-uncased_test.npy (deflated 10%)
  adding: repr/roberta-base_validation.npy (deflated 8%)
  adding: repr/roberta-large_train.npy (deflated 8%)
  adding: repr/roberta-large_test.npy (deflated 8%)
  adding: repr/bert-base-uncased_train.npy (deflated 10%)
  adding: repr/bert-large-uncased_validation.npy (deflated 10%)
  adding: repr/roberta-large_validation.npy (deflated 8%)
